In [1]:
! pip install --upgrade pymilvus requests tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.6
    Uninstalling tqdm-4.66.6:
      Successfully uninstalled tqdm-4.66.6


### Prepare the data

In [2]:
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
from glob import glob

text_lines = []

for file_path in glob("/content/drive/MyDrive/Saudi_Arabia_Landmarks_Dataset/Data/text_data/*.txt", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

In [12]:
text_lines

['"في قلب مدينة جدة النابضة بالحياة، حيث يلتقي البحر بأحضان المدينة، يبرز مسجد الرحمة كواحد من أبرز المعالم الإسلامية. يتموقع هذا المسجد على شاطئ البحر الأحمر، كأنه يشمخ بفخر أمام أمواج البحر التي تعانق ضفافه.وقد تم تصميمه بأسلوب معماري مميز يمزج بين الطراز الإسلامي الكلاسيكي والعصري، مما يجعله ليس مجرد مكان للصلاة، بل تحفة فنية تنطق بأصالة العمارة الإسلامية. يزين المسجد قبة رائعة تتلألأ في ضوء الشمس، وأعمدة تروي حكايات من الإيمان والهدوء.لكن ما يميز مسجد الرحمة حقًا هو موقعه الفريد، حيث يمكن للزوار والمصلين الاستمتاع بأجواء البحر وهو يتلاطم بأمواجه على الصخور. في كل صلاة، تكتمل الصورة الجميلة بأصوات التكبير والأذان، والتي تتناغم مع أصوات الأمواج وكأنها دعوة للروح للانغماس في سكون البحر وعمق الإيمان.\n\nليس فقط مكانًا للعبادة، بل هو أيضًا ملتقى ثقافي واجتماعي. يتوافد إليه الزوار من مختلف الأعمار، ليشهدوا جمال الهندسة المعمارية، ويستمتعوا بمساحات الجلوس الخارجية التي تطل على البحر. في المساء، يتلألأ المسجد بأضوائه، ليخلق جوًا من السحر والسكينة."\t',
 '"في قلب مدينة الرياض القديمة، يقع ق

### Prepare the Embedding Model

In [13]:
from sentence_transformers import SentenceTransformer

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [14]:
# Load the embedding model only once for efficiency
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def emb_text(text):
    """
    Generate embeddings for a single text input using SentenceTransformer.

    Args:
    text (str): The input text for which embeddings are to be generated.

    Returns:
    list: The generated embedding as a list.
    """
    embedding = model.encode(text).tolist()  # Convert embedding to list for Milvus
    return embedding

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Generate a test embedding and print its dimension and first few elements.

In [15]:
test_embedding = emb_text("برج المملكة")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

384
[0.08536312729120255, 0.39538663625717163, -0.0432596281170845, -0.2185550481081009, -0.2143644243478775, -0.5810228586196899, -0.1615285575389862, 0.03853974863886833, 0.14944110810756683, -0.0022624267730861902]


## Load data into Milvus

### Create the Collection

In [16]:
from tqdm.autonotebook import tqdm

In [17]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "saudi_landmarks"

DEBUG:pymilvus.milvus_client.milvus_client:Created new connection using: b4581bf1234a423d8793ba88cc124b46


> As for the argument of `MilvusClient`:
> - Setting the `uri` as a local file, e.g.`./milvus.db`, is the most convenient method, as it automatically utilizes [Milvus Lite](https://milvus.io/docs/milvus_lite.md) to store all data in this file.
> - If you have large scale of data, you can set up a more performant Milvus server on [docker or kubernetes](https://milvus.io/docs/quickstart.md). In this setup, please use the server uri, e.g.`http://localhost:19530`, as your `uri`.
> - If you want to use [Zilliz Cloud](https://zilliz.com/cloud), the fully managed cloud service for Milvus, adjust the `uri` and `token`, which correspond to the [Public Endpoint and Api key](https://docs.zilliz.com/docs/on-zilliz-cloud-console#free-cluster-details) in Zilliz Cloud.

Check if the collection already exists and drop it if it does.

In [18]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)

Create a new collection with specified parameters.

If we don't specify any field information, Milvus will automatically create a default `id` field for primary key, and a `vector` field to store the vector data. A reserved JSON field is used to store non-schema-defined fields and their values.

In [19]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="L2",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)

DEBUG:pymilvus.milvus_client.milvus_client:Successfully created collection: saudi_landmarks
DEBUG:pymilvus.milvus_client.milvus_client:Successfully created an index on collection: saudi_landmarks


### Insert data
Iterate through the text lines, create embeddings, and then insert the data into Milvus.

Here is a new field `text`, which is a non-defined field in the collection schema. It will be automatically added to the reserved JSON dynamic field, which can be treated as a normal field at a high level.

In [20]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data)

Creating embeddings: 100%|██████████| 16/16 [00:02<00:00,  5.93it/s]


{'insert_count': 16, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 'cost': 0}

## Build RAG

### Retrieve data for a query

Let's specify a frequent question about Milvus.

In [21]:
question = "قصر المصمك"

Search for the question in the collection and retrieve the semantic top-3 matches.

In [22]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=[
        emb_text(question)
    ],  # Use the `emb_text` function to convert the question to an embedding vector
    limit=3,  # Return top 3 results
    search_params={"metric_type": "L2", "params": {"nprobe": 10}}, # Euclidean distance
    output_fields=["text"],  # Return the text field
)

In [23]:
search_res

data: ['[{\'id\': 1, \'distance\': 8.09587574005127, \'entity\': {\'text\': \'"في قلب مدينة الرياض القديمة، يقع قصر المصمك، الحصن الضخم الذي شهد لحظة تاريخية مفصلية في تاريخ المملكة العربية السعودية. بُني القصر في عام 1865 على يد الأمير عبد الله بن فيصل بن تركي كحصن دفاعي لحماية المدينة، ويعود اسمه إلى كلمة “المصمك” التي تعني البناء القوي والحصين.\\n\\nلكن القصة التي جعلت من قصر المصمك رمزًا خالدًا في تاريخ المملكة، حدثت في فجر يوم مشرق من عام 1902. في ذلك اليوم، استطاع الملك عبد العزيز بن عبد الرحمن آل سعود، بعد فترة من النفي، استعادة الرياض والسيطرة على القصر في معركة حاسمة ضد آل رشيد. كانت هذه اللحظة نقطة التحول الكبرى في تأسيس الدولة السعودية الحديثة، حيث أصبح قصر المصمك بمثابة رمز للوحدة والقوة.\\n\\nعند دخولك إلى القصر اليوم، يمكن أن تشعر بأصداء الماضي تتردد في أرجاءه. ستجد الأبراج العالية المصنوعة من الطين والحجارة، والتي كانت تستخدم للدفاع، والبوابة الخشبية الضخمة التي لا تزال تحتفظ بآثار المعركة التي دارت عندها.\\n\\nالآن، تم تحويل قصر المصمك إلى متحف يعرض مراحل توحيد المملكة،

Let's take a look at the search results of the query


In [24]:
search_res[0]

[{'id': 1,
  'distance': 8.09587574005127,
  'entity': {'text': '"في قلب مدينة الرياض القديمة، يقع قصر المصمك، الحصن الضخم الذي شهد لحظة تاريخية مفصلية في تاريخ المملكة العربية السعودية. بُني القصر في عام 1865 على يد الأمير عبد الله بن فيصل بن تركي كحصن دفاعي لحماية المدينة، ويعود اسمه إلى كلمة “المصمك” التي تعني البناء القوي والحصين.\n\nلكن القصة التي جعلت من قصر المصمك رمزًا خالدًا في تاريخ المملكة، حدثت في فجر يوم مشرق من عام 1902. في ذلك اليوم، استطاع الملك عبد العزيز بن عبد الرحمن آل سعود، بعد فترة من النفي، استعادة الرياض والسيطرة على القصر في معركة حاسمة ضد آل رشيد. كانت هذه اللحظة نقطة التحول الكبرى في تأسيس الدولة السعودية الحديثة، حيث أصبح قصر المصمك بمثابة رمز للوحدة والقوة.\n\nعند دخولك إلى القصر اليوم، يمكن أن تشعر بأصداء الماضي تتردد في أرجاءه. ستجد الأبراج العالية المصنوعة من الطين والحجارة، والتي كانت تستخدم للدفاع، والبوابة الخشبية الضخمة التي لا تزال تحتفظ بآثار المعركة التي دارت عندها.\n\nالآن، تم تحويل قصر المصمك إلى متحف يعرض مراحل توحيد المملكة، ويستقطب الزوار من 

In [25]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4, ensure_ascii=False))

[
    [
        "\"في قلب مدينة الرياض القديمة، يقع قصر المصمك، الحصن الضخم الذي شهد لحظة تاريخية مفصلية في تاريخ المملكة العربية السعودية. بُني القصر في عام 1865 على يد الأمير عبد الله بن فيصل بن تركي كحصن دفاعي لحماية المدينة، ويعود اسمه إلى كلمة “المصمك” التي تعني البناء القوي والحصين.\n\nلكن القصة التي جعلت من قصر المصمك رمزًا خالدًا في تاريخ المملكة، حدثت في فجر يوم مشرق من عام 1902. في ذلك اليوم، استطاع الملك عبد العزيز بن عبد الرحمن آل سعود، بعد فترة من النفي، استعادة الرياض والسيطرة على القصر في معركة حاسمة ضد آل رشيد. كانت هذه اللحظة نقطة التحول الكبرى في تأسيس الدولة السعودية الحديثة، حيث أصبح قصر المصمك بمثابة رمز للوحدة والقوة.\n\nعند دخولك إلى القصر اليوم، يمكن أن تشعر بأصداء الماضي تتردد في أرجاءه. ستجد الأبراج العالية المصنوعة من الطين والحجارة، والتي كانت تستخدم للدفاع، والبوابة الخشبية الضخمة التي لا تزال تحتفظ بآثار المعركة التي دارت عندها.\n\nالآن، تم تحويل قصر المصمك إلى متحف يعرض مراحل توحيد المملكة، ويستقطب الزوار من جميع أنحاء العالم لاستكشاف تاريخ المملكة العريق

In [26]:
context = retrieved_lines_with_distances[0][0]

In [27]:
context

'"في قلب مدينة الرياض القديمة، يقع قصر المصمك، الحصن الضخم الذي شهد لحظة تاريخية مفصلية في تاريخ المملكة العربية السعودية. بُني القصر في عام 1865 على يد الأمير عبد الله بن فيصل بن تركي كحصن دفاعي لحماية المدينة، ويعود اسمه إلى كلمة “المصمك” التي تعني البناء القوي والحصين.\n\nلكن القصة التي جعلت من قصر المصمك رمزًا خالدًا في تاريخ المملكة، حدثت في فجر يوم مشرق من عام 1902. في ذلك اليوم، استطاع الملك عبد العزيز بن عبد الرحمن آل سعود، بعد فترة من النفي، استعادة الرياض والسيطرة على القصر في معركة حاسمة ضد آل رشيد. كانت هذه اللحظة نقطة التحول الكبرى في تأسيس الدولة السعودية الحديثة، حيث أصبح قصر المصمك بمثابة رمز للوحدة والقوة.\n\nعند دخولك إلى القصر اليوم، يمكن أن تشعر بأصداء الماضي تتردد في أرجاءه. ستجد الأبراج العالية المصنوعة من الطين والحجارة، والتي كانت تستخدم للدفاع، والبوابة الخشبية الضخمة التي لا تزال تحتفظ بآثار المعركة التي دارت عندها.\n\nالآن، تم تحويل قصر المصمك إلى متحف يعرض مراحل توحيد المملكة، ويستقطب الزوار من جميع أنحاء العالم لاستكشاف تاريخ المملكة العريق. يعرض المتحف مقت

In [34]:
import numpy as np

In [28]:
# Directory containing your .txt files
directory = "/content/drive/MyDrive/Saudi_Arabia_Landmarks_Dataset/Data/text_data"

# List all .txt files in the directory
txt_files = [f for f in os.listdir(directory) if f.endswith(".txt")]

# Define a list of your 16 questions
questions = []

# Translate and rename each file
for filename in txt_files:
    questions.append(filename)


In [48]:
# Collect all similarity scores
similarity_scores = []

for question in questions:
    # Convert the question to an embedding
    question_embedding = emb_text(question)

    # Search in Milvus for top 3 most similar documents
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=[question_embedding],
        limit=3,
        search_params={"metric_type": "L2", "params": {"nprobe": 10}},
        output_fields=["text"]
    )

    # Calculate similarity for each result (e.g., based on Euclidean distance)
    question_similarities = []
    for result in search_res:
        distance = result[0]['distance']  # Assuming Milvus provides `distance` in the result
        similarity = 1 / (1 + distance)  # Convert L2 distance to similarity score
        question_similarities.append(similarity)

    # Take the mean similarity for the current question
    mean_similarity = np.mean(question_similarities)
    similarity_scores.append(mean_similarity)

# Calculate the overall mean similarity across all questions
overall_mean_similarity = np.mean(similarity_scores)

print(f"Overall Mean Similarity across 16 questions: {1-overall_mean_similarity}")

Overall Mean Similarity across 16 questions: 0.9174216145715577
